# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

# def gradf(a, b, c):
#   return [0, 0, 0] # todo, return [df/da, df/db, df/dc]

# # expected answer is the list of
# ans = [-12.353553390593273, 10.25699027111255, 0.0625]
# yours = gradf(2, 3, 4)
# for dim in range(3):
#   ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
#   print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

def gradf(a, b, c):
  dfa = -3*a**2 - 0.5*a**-0.5
  dfb = 3*cos(3*b) + 2.5*b**1.5
  dfc = 1/c**2
  return [dfa, dfb, dfc]

ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")



OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# # -----------
# numerical_grad = [0, 0, 0] # TODO
# # -----------

# for dim in range(3):
#   ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
#   print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

h = 1e-5
numerical_grad = [0, 0, 0]

def f_partial_derivative(f, vars, i, h):
    vars_h1 = vars[:]
    vars_h2 = vars[:]
    vars_h1[i] += h
    vars_h2[i] -= h
    return (f(*vars_h1) - f(*vars_h2)) / (2 * h)

a, b, c = 2, 3, 4
variables = [a, b, c]

for i in range(3):
    numerical_grad[i] = f_partial_derivative(f, variables, i, h)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# # -----------
# numerical_grad2 = [0, 0, 0] # TODO
# # -----------

# for dim in range(3):
#   ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
#   print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

numerical_grad2 = [0, 0, 0]

for i in range(3):
    numerical_grad2[i] = f_partial_derivative(f, variables, i, h)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


## section 2: support for softmax

In [ ]:
# # Value class starter code, with many functions taken out
# from math import exp, log

# class Value:

#   def __init__(self, data, _children=(), _op='', label=''):
#     self.data = data
#     self.grad = 0.0
#     self._backward = lambda: None
#     self._prev = set(_children)
#     self._op = _op
#     self.label = label

#   def __repr__(self):
#     return f"Value(data={self.data})"

#   def __add__(self, other): # exactly as in the video
#     other = other if isinstance(other, Value) else Value(other)
#     out = Value(self.data + other.data, (self, other), '+')

#     def _backward():
#       self.grad += 1.0 * out.grad
#       other.grad += 1.0 * out.grad
#     out._backward = _backward

#     return out

#   # ------
#   # re-implement all the other functions needed for the exercises below
#   # your code here
#   # TODO
#   # ------

#   def backward(self): # exactly as in video
#     topo = []
#     visited = set()
#     def build_topo(v):
#       if v not in visited:
#         visited.add(v)
#         for child in v._prev:
#           build_topo(child)
#         topo.append(v)
#     build_topo(self)

#     self.grad = 1.0
#     for node in reversed(topo):
#       node._backward()

from math import exp, log, sin, cos, sqrt

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data / other.data, (self, other), '/')

        def _backward():
            self.grad += (1 / other.data) * out.grad
            other.grad += (-self.data / (other.data ** 2)) * out.grad
        out._backward = _backward

        return out

    def __neg__(self):
        out = Value(-self.data, (self,), 'neg')

        def _backward():
            self.grad += -out.grad
        out._backward = _backward

        return out

    def exp(self):
        out = Value(exp(self.data), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def log(self):
        out = Value(log(self.data), (self,), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __rtruediv__(self, other):
        return Value(other) / self

    def __rpow__(self, other):
        return Value(other) ** self

def softmax(logits):
    exp_logits = [logit.exp() for logit in logits]
    sum_exp_logits = sum(exp_logits)
    probs = [exp_logit / sum_exp_logits for exp_logit in exp_logits]
    return probs


logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log()  # Compute loss using negative log likelihood

# Displaying computed gradients
print("Computed loss:", loss.data)
for i, logit in enumerate(logits):
    print(f"Gradient of logit {i}: {logit.grad}")

# Verification with PyTorch
import torch

logits_torch = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
probs_torch = torch.nn.functional.softmax(logits_torch, dim=0)
loss_torch = -torch.log(probs_torch[3])
loss_torch.backward()

print("PyTorch gradients:")
for i in range(4):
    print(f"Gradient of logit {i}: {logits_torch.grad[i].item()}")
ans = [logits_torch.grad[i].item() for i in range(4)]

for dim in range(4):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


Computed loss: 2.1755153626167147
Gradient of logit 0: 0.0
Gradient of logit 1: 0.0
Gradient of logit 2: 0.0
Gradient of logit 3: 0.0
PyTorch gradients:
Gradient of logit 0: 0.041772566735744476
Gradient of logit 1: 0.8390244841575623
Gradient of logit 2: 0.005653302650898695
Gradient of logit 3: -0.8864504098892212
WRONG! for dim 0: expected 0.041772566735744476, yours returns 0.0
WRONG! for dim 1: expected 0.8390244841575623, yours returns 0.0
WRONG! for dim 2: expected 0.005653302650898695, yours returns 0.0
WRONG! for dim 3: expected -0.8864504098892212, yours returns 0.0


In [ ]:
# # verify the gradient using the torch library
# # torch should give you the exact same gradient
# import torch

import torch

logits_torch = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
probs_torch = torch.nn.functional.softmax(logits_torch, dim=0)
loss_torch = -torch.log(probs_torch[3])
loss_torch.backward()

print("PyTorch gradients:")
for i in range(4):
    print(f"Gradient of logit {i}: {logits_torch.grad[i].item()}")
ans = [logits_torch.grad[i].item() for i in range(4)]
for dim in range(4):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


PyTorch gradients:
Gradient of logit 0: 0.041772566735744476
Gradient of logit 1: 0.8390244841575623
Gradient of logit 2: 0.005653302650898695
Gradient of logit 3: -0.8864504098892212
WRONG! for dim 0: expected 0.041772566735744476, yours returns 0.0
WRONG! for dim 1: expected 0.8390244841575623, yours returns 0.0
WRONG! for dim 2: expected 0.005653302650898695, yours returns 0.0
WRONG! for dim 3: expected -0.8864504098892212, yours returns 0.0


In [ ]:
# very dispointed